In [3]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4 #beatifulsoup4
import json

# Colocando o assunto que deseja recomendar
Decidimos as palavras chaves / assuntos que será pesquisado.
<br> Descobrimos por pesquisa rápida, algumas 'tags' do URL, que nos permite pesquisar novos envios e páginas do resultado da pesquisa (como "sp=CAI%253D&" e "p=1")

In [4]:
queries = ["machine+learning", "data_science", "kanggle"]
url = "https://www.youtube.com/results?search_query={query}&sp=CAI%253D&p={page}"

# Coleta dos dados
Comentarei linha a linha explicando suas funções

In [6]:
# repetição para todos as palavras chaves pesquisados
for query in queries:
    # número de páginas da pesquisa por palavra chave
    for page in range(1, 6):
        # monta o URL das páginas e printa o mesmo, conforme saída
        urll = url.format(query = query, page = page)
        print(urll)
        # busca o código fonte das páginas (links) montados anteriormente
        response = rq.get(urll)
        # cria arquivos (.html) dessas páginas em uma pasta dentro do projeto com nome (dados_brutos)
        with open("./dados_brutos/{}_{}.html".format(query, page), "w+", encoding="utf-8") as output:
            output.write(response.text)
        # repetição ocorre a cada dois segundos, para caso o youtube não bloqueie a resposta    
        time.sleep(2)  

https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=1
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=2
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=3
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=4
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=5
https://www.youtube.com/results?search_query=data_science&sp=CAI%253D&p=1
https://www.youtube.com/results?search_query=data_science&sp=CAI%253D&p=2
https://www.youtube.com/results?search_query=data_science&sp=CAI%253D&p=3
https://www.youtube.com/results?search_query=data_science&sp=CAI%253D&p=4
https://www.youtube.com/results?search_query=data_science&sp=CAI%253D&p=5
https://www.youtube.com/results?search_query=kanggle&sp=CAI%253D&p=1
https://www.youtube.com/results?search_query=kanggle&sp=CAI%253D&p=2
https://www.youtube.com/results?search_query=kanggle&sp=CAI%253D&p=3
https://www.youtube.com/results?s

# Processamento dados brutos
Comentarei linha a linha explicando suas funções

In [7]:
# repetição para todos as palavras chaves pesquisados
for query in queries:
    # número de páginas da pesquisa por palavra chave
    for page in range(1, 6):
        with open("./dados_brutos/{}_{}.html".format(query, page), "r+", encoding="utf-8") as inp:
            # lendo e atribuindo arquivos .html salvos anteriormente
            page_html = inp.read()
            # utilizo a função para receber o código fonte da página
            parsed = bs4.BeautifulSoup(page_html)
            # por inspeção da página manualmente, descubro que os links e titulos do vídeo estão SEMPRE dentro de uma tag <a>.
            # porém existem outras coisas armazenadas (sem ser o nosso objetivo), utilizando essa mesma tag html.
            tags = parsed.findAll("a")
            # na inspeção realizada anteriormente, descobrimos que todas as tag "<a>" que desejamos obter, tem uma tag dentro
            # dela cujo o nome é "aria-describedby", por isso esse looping serve para procurar essas tags.
            for e in tags:
                # verifica se é a tag pesquisada
                if e.has_attr("aria-describedby"):
                    # recebe o link e o título, que é informações desejadas.
                    link = e['href']
                    title = e['title']
                    # cria um arquivo .json, cujo cada linha significa 1 vídeo com as seguintes informações:
                    # Link, título e assunto pesquisado
                    with open("parsed_videos.json", "a+", encoding="utf-8") as output:
                        data = {"link": link, "title": title, "query": query}
                        output.write("{}\n".format(json.dumps(data)))
                    
        

# Verificação do resultado
Coloca os vídeos salvos no arquivo Json em um DataFrame

In [8]:
df = pd.read_json("parsed_videos.json", lines=True)

In [9]:
df.sort_values("title")

,link,title,query
404,/watch?v=OoB_LQpgDCk,"""Inversion"": Walter Reade | Data Science at Ka...",kanggle
188,/watch?v=OoB_LQpgDCk,"""Inversion"": Walter Reade | Data Science at Ka...",kanggle
195,/watch?v=uVHjY7owVl0,#Kaggle #COVID19 #Data-Science-Diva Walkthroug...,kanggle
411,/watch?v=uVHjY7owVl0,#Kaggle #COVID19 #Data-Science-Diva Walkthroug...,kanggle
376,/watch?v=7xHjlpjWm8Q,"#task-vt - Apr 12, 2020 - sync on kaggle submi...",kanggle
...,...,...,...
125,/watch?v=rYoRhIuke-U,kaggle PANDA competition video 4 CNN model - I...,kanggle
294,/watch?v=iTc9URy_wzg,theTechTalk #9: Machine Learning: Beyond Myths...,machine+learning
69,/watch?v=iTc9URy_wzg,theTechTalk #9: Machine Learning: Beyond Myths...,machine+learning
260,/watch?v=BRZZt_0v5JM,▷ Machine Learning no MS Excel - KNN e K-Means,machine+learning
